In [ ]:
import pandas as pd
import numpy as np
import re
import os

FILE_EXCEL_PATH = "data/Database-Q3_2020.xlsx"
SHEET_MKT = "MKT"
SHEET_SALES = "Sales"
SHEET_VANDON = "Vận đơn"

def clean_column_names(df):
    """Chuẩn hóa tên cột."""
    df = df.copy()
    new_cols = []
    patterns = {
        '[àáâãăạảấầẩẫậắằẳẵặ]': 'a', '[èéêẹẻẽếềểễệ]': 'e',
        '[ìíịỉĩ]': 'i', '[òóôõơọỏốồổỗộớờởỡợ]': 'o',
        '[ùúụủũưứừửữự]': 'u', '[ỳýỵỷỹ]': 'y', '[đ]': 'd'
    }
    for col in df.columns:
        col_str = str(col).lower().strip()
        col_str = re.sub(r'[\n\r]+', ' ', col_str)
        for regex, replacement in patterns.items():
            col_str = re.sub(regex, replacement, col_str)
        col_str = re.sub(r'[^a-z0-9\s]', '', col_str)
        col_str = re.sub(r'\s+', '_', col_str)
        new_cols.append(col_str)
    df.columns = new_cols
    return df

def fix_year_logic(date_val, target_year=2020):
    if pd.isna(date_val): return pd.NaT
    return date_val.replace(year=target_year) if date_val.year == 2023 else date_val

def process_excel_data():
    try:
        print(f"Đọc file Excel: {FILE_EXCEL_PATH}")
        if not os.path.exists(FILE_EXCEL_PATH):
            print(f"LỖI: Không tìm thấy file '{FILE_EXCEL_PATH}'")
            return

        xls = pd.ExcelFile(FILE_EXCEL_PATH)
        
        try:
            df_mkt = pd.read_excel(xls, sheet_name=SHEET_MKT)
            df_sales = pd.read_excel(xls, sheet_name=SHEET_SALES)
            df_van_don = pd.read_excel(xls, sheet_name=SHEET_VANDON)
        except Exception as e:
            print(f"Lỗi đọc sheet: {e}")
            return

        print("Xử lý MKT")
        df_mkt = clean_column_names(df_mkt)
        df_mkt['date'] = pd.to_datetime(df_mkt['date'], errors='coerce')
        df_mkt_clean = df_mkt[
            (df_mkt['date'].dt.year == 2020) & (df_mkt['date'].dt.quarter == 3)
        ].copy()
        
        cols_numeric_mkt = ['chi_phi_marketing', 'lead_mkt', 'don_hang', 'doanh_thu', 'inbox', 'comment']
        for col in cols_numeric_mkt:
            if col in df_mkt_clean.columns:
                df_mkt_clean[col] = pd.to_numeric(df_mkt_clean[col], errors='coerce').fillna(0)

        print("Xử lý Sales")
        df_sales = clean_column_names(df_sales)
        if len(df_sales.columns) > 0:
            col_lead_date = df_sales.columns[0]
            df_sales.rename(columns={col_lead_date: 'lead_date'}, inplace=True)
            df_sales['lead_date'] = pd.to_datetime(df_sales['lead_date'], dayfirst=True, errors='coerce')
            df_sales_clean = df_sales[
                (df_sales['lead_date'].dt.year == 2020) & (df_sales['lead_date'].dt.quarter == 3)
            ].copy()
        else:
            df_sales_clean = pd.DataFrame()

        if 'close_date' in df_sales_clean.columns:
            df_sales_clean['close_date'] = pd.to_datetime(df_sales_clean['close_date'], dayfirst=True, errors='coerce')
            df_sales_clean['close_date'] = df_sales_clean['close_date'].apply(fix_year_logic)

        if not df_sales_clean.empty:
            trang_thai = df_sales_clean['trang_thai'].fillna('').astype(str)
            level = df_sales_clean['level'].fillna('').astype(str)
            df_sales_clean['is_successful'] = (
                trang_thai.str.contains('dat hang', case=False, regex=False) | 
                level.str.contains('l8', case=False, regex=False)
            )
            if 'tong_tien' in df_sales_clean.columns:
                df_sales_clean['tong_tien'] = pd.to_numeric(df_sales_clean['tong_tien'], errors='coerce').fillna(0)

        print("Xử lý Vận Đơn")
        df_van_don = clean_column_names(df_van_don)
        
        # Xử lý ngày tháng
        date_cols_vandon = ['ngay_dong_goi', 'ngay_xuat_kho', 'ngay_giao_hang']
        for col in date_cols_vandon:
            if col in df_van_don.columns:
                df_van_don[col] = pd.to_datetime(df_van_don[col], errors='coerce')

        if 'ngay_dong_goi' in df_van_don.columns:
            df_van_don_clean = df_van_don[
                (df_van_don['ngay_dong_goi'].dt.year == 2020) & 
                (df_van_don['ngay_dong_goi'].dt.quarter == 3)
            ].copy()
            
            if 'sdt_nguoi_nhan' in df_van_don_clean.columns:
                print("   > Đang làm sạch cột 'sdt_nguoi_nhan'...")
                df_van_don_clean['sdt_nguoi_nhan'] = df_van_don_clean['sdt_nguoi_nhan'].astype(str)
                df_van_don_clean['sdt_nguoi_nhan'] = df_van_don_clean['sdt_nguoi_nhan'].str.replace(r'\D', '', regex=True)
                df_van_don_clean.loc[df_van_don_clean['sdt_nguoi_nhan'] == 'nan', 'sdt_nguoi_nhan'] = ''

            cols_numeric_vandon = ['tong_tien_hang', 'phi_van_chuyen', 'tong_tien_thu_ho']
            for col in cols_numeric_vandon:
                if col in df_van_don_clean.columns:
                    df_van_don_clean[col] = pd.to_numeric(df_van_don_clean[col], errors='coerce').fillna(0)
        else:
            df_van_don_clean = pd.DataFrame()

        print("Xuất CSV:.")
        df_mkt_clean.to_csv('data_pbi/Processed_MKT_Q3.csv', index=False, encoding='utf-8-sig')
        df_sales_clean.to_csv('data_pbi/Processed_Sales_Q3.csv', index=False, encoding='utf-8-sig')
        df_van_don_clean.to_csv('data_pbi/Processed_VanDon_Q3.csv', index=False, encoding='utf-8-sig')
        
        print(f"Số dòng: MKT={len(df_mkt_clean)}, Sales={len(df_sales_clean)}, Vận Đơn={len(df_van_don_clean)}")

    except Exception as e:
        print(f"LỖI: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    process_excel_data()

Đang đọc file Excel: Database-Q3_2020.xlsx ...
--- [1/3] Xử lý MKT ---
--- [2/3] Xử lý Sales ---
--- [3/3] Xử lý Vận Đơn ---
   > Đang làm sạch cột 'sdt_nguoi_nhan'...
Đang xuất file CSV...
Hoàn tất! Số dòng: MKT=1511, Sales=9504, Vận Đơn=5820
